In [12]:
import pandas as pd
import requests
import numpy as np
from urlib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup

In [13]:
url = 'https://www.fundsexplorer.com.br/ranking'
headers = {
    'User-Agent': 
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36'
        ' (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
}

In [15]:
response = requests.get(url, headers=headers)
html = response.content
dados = BeautifulSoup(html, "html.parser")
if response.status_code == 200:
    df = pd.read_html(response.content, encoding='utf-8')[0]
else:
    print("Falha na chamada da API")

In [7]:
df.sort_values('Fundo', inplace=True)

KeyError: 'Fundo'

In [16]:
dados

<!DOCTYPE html>

<html class="" lang="pt-BR">
<head>
<!-- AdOpt -->
<!-- <meta name="adopt-website-id" content="2e16f44d-7122-44e3-823d-3153536f3017" />
    <script src="//tag.goadopt.io/injector.js?website_code=2e16f44d-7122-44e3-823d-3153536f3017" class="adopt-injector"></script> -->
<!-- End AdOpt -->
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport">
<link href="https://www.fundsexplorer.com.br/wp-content/themes/fundsexplorer/resources/images/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="https://www.fundsexplorer.com.br/wp-content/themes/fundsexplorer/resources/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
<!-- Checkout Cookie ID -->
<script src="https://www.fundsexplorer.com.br/wp-content/plugins/suno-checkoutcookie/assets/js/frontend.js" type="text/

In [9]:
response

<Response [200]>